In [1]:
# Requirements and imports

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time
from catboost import CatBoostClassifier
from catboost import CatBoostClassifier, Pool, cv

In [2]:
# Abundance Matrix

df_bahram = pd.read_csv('data/df_bahramSamp.tsv', sep='\t', encoding='utf-8')
df_bahram.head()

/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,SampleID,Sample_accnos,Sample Accession,Habitat_type,Continent,Total_reads,Rejected1,Rejected2,Accepted1,...,Bacteria;Actinobacteria;Acidimicrobiia;Acidimicrobiales;Sva0996 marine group;uncultured Cellulomonadaceae bacterium,Bacteria;Bacteroidetes;Bacteroidetes Incertae Sedis;Order III;Unknown Family;Bacteroidetes bacterium MO48,Bacteria;Proteobacteria;Gammaproteobacteria;Chromatiales;Chromatiaceae;Thiophaeococcus sp. JA592,Bacteria;Cyanobacteria;Chloroplast;Ranunculus sp. Qiu 95024;?;?,Bacteria;Firmicutes;Bacilli;Bacillales;anaerobic thermophile IC-BH;?,Bacteria;Actinobacteria;Actinobacteria;Micrococcales;Cellulomonadaceae;actinobacterium PY,Bacteria;Nitrospirae;Nitrospira;Nitrospirales;BMS9AB35;uncultured delta proteobacterium GR-296.I.86,Bacteria;Bacteroidetes;Cytophagia;Cytophagales;Cyclobacteriaceae;Mongoliibacter,Bacteria;Proteobacteria;Gammaproteobacteria;KI89A clade;gamma proteobacterium HdN1;?,Bacteria;Cyanobacteria;Cyanobacteria;SubsectionIV;FamilyI;Aphanizomenon flos-aquae 1477
0,0,GC1,SAMEA103892432,SAMEA103892432,Soil,Asia,10953226,3557232,2486229,3564892,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
1,1,GC10,SAMEA103892441,SAMEA103892441,Soil,South_America,2064028,1224211,0,839817,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
2,2,GC100,SAMEA103892529,SAMEA103892529,Soil,South_Europe,1532826,941342,0,591484,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
3,3,GC102,SAMEA103892531,SAMEA103892531,Soil,West_Asia,1964974,1361843,0,603131,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
4,4,GC103,SAMEA103892534,SAMEA103892534,Soil,Central_America,4996050,3056576,11,1939462,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0


In [3]:
# How many vals are null?
null_value_stats = df_bahram.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Bacteria_to_fungi_CAZyme_ratio    2239
dtype: int64

In [4]:
df_sample_attributes = pd.read_csv('data/df_sampleAttributes.tsv', sep='\t', encoding='utf-8')
df_sample_attributes.head()

/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (60,75,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,SRA accession,Title,Project name,Study name,Study design,Description,Collection date,Investigation Type,Sample name,...,Total organic carbon method,Water content method,Water content,Total nitrogen,Total organic carbon,Latitude Start,Longitude Start,Latitude End,Longitude End,Lat tag
0,SAMEA2338835,ERS396505,Stool sample from spanish,MetaHIT-IGC,NaN,NaN,NaN,2008/2010,metagenome,ERS396505,...,NaN,NaN,NaN,NaN,NaN,40.416633,-3.703766,NaN,NaN,geographic location (latitude)
1,SAMEA3920725,ERS1107859,Processed metagenomic and transcriptomic activ...,What is the role of biodiversity on the functi...,NaN,NaN,NaN,2014,metagenome,ERS1107859,...,NaN,NaN,NaN,NaN,NaN,47.404830,8.609670,NaN,NaN,geographic location (latitude)
2,SAMEA1879404,ERS238577,Rhizosphere community metatranscriptomics,Rhizosphere metatranscriptomics,NaN,NaN,Analysis of community structure in the rhizosp...,2009-03,metagenome,ERS238577,...,NaN,NaN,NaN,NaN,0.23,53.041389,1.372500,NaN,NaN,geographic location (latitude)
3,SAMEA3663100,ERS970249,Stool sample from pig strain : LW,A Catalogue of the pig Gut Metagenome,NaN,NaN,Large White,2012,metagenome,ERS970249,...,NaN,NaN,NaN,NaN,NaN,46.104913,-0.751411,NaN,NaN,geographic location (latitude)
4,SAMN00067299,NaN,NaN,NaN,HMP Core Microbiome Sampling Protocol A (HMP-A),Control Set,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
nzeros = [i for i,v in enumerate(df_bahram.iloc[:,21:].sum()==0.0) if v==False]
columns = list(df_bahram.iloc[:,0:21].columns)+list(df_bahram.iloc[:,21:].iloc[:,nzeros].columns)
nzeros = [i for i,v in enumerate(df_bahram.iloc[:,21:].sum(axis=1)==0.0) if v==False]
indices = list(df_bahram.iloc[nzeros,21:].index)
df_bahramZ = df_bahram[columns].loc[indices]
df_bahramZ['Habitat_type'].value_counts()

Human_Gut       7202
Human_Oral      1585
Built           1190
Human_Skin      1126
Soil             502
Wastewater       300
Pig_Gut          295
Mouse_Gut        215
Human_Nose       195
Marine           130
Human_Vagina     125
Dog_Gut          124
Cat_Gut          124
Fresh_Water      104
Birds             77
Dairy             35
Fermented         26
Bovine_Rumen      26
Food              25
Composting        17
Root              14
Porifera          12
Phylloplane        7
Hot_spring         3
Fish_Skin          2
Name: Habitat_type, dtype: int64

In [7]:
y = df_bahramZ['Habitat_type']
y

0            Soil
1            Soil
2            Soil
3            Soil
4            Soil
           ...   
13478    Porifera
13479    Porifera
13480    Porifera
13481    Porifera
13482    Porifera
Name: Habitat_type, Length: 13461, dtype: object

In [8]:
X = df_bahramZ.iloc[:,21:]
# Let's put the X's on a common scale
scaler = MinMaxScaler()
X[X.columns] = scaler.fit_transform(X)
# Let's make the categories ordinal
habitat_ord={h:i for i,h in enumerate(set(y))}
inv_map = {v: k for k, v in habitat_ord.items()}
X

,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Bacteroidaceae;Bacteroides,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella 9,Bacteria;Firmicutes;Clostridia;Clostridiales;Lachnospiraceae;?,Bacteria;Firmicutes;Clostridia;Clostridiales;Ruminococcaceae;Faecalibacterium,Bacteria;Firmicutes;Bacilli;Lactobacillales;Streptococcaceae;Streptococcus,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacteriales;Enterobacteriaceae;Escherichia-Shigella,Bacteria;Actinobacteria;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Bifidobacterium,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacteriales;Enterobacteriaceae;?,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Pseudomonadaceae;Pseudomonas,Bacteria;Firmicutes;Bacilli;Lactobacillales;Lactobacillaceae;Lactobacillus,...,Bacteria;Proteobacteria;Deltaproteobacteria;NB1-j;uncultured gamma proteobacterium;?,Bacteria;Cyanobacteria;Cyanobacteria;SubsectionIV;FamilyI;Anabaena sp. BECID8,Bacteria;Actinobacteria;Acidimicrobiia;Acidimicrobiales;Sva0996 marine group;uncultured Cellulomonadaceae bacterium,Bacteria;Bacteroidetes;Bacteroidetes Incertae Sedis;Order III;Unknown Family;Bacteroidetes bacterium MO48,Bacteria;Proteobacteria;Gammaproteobacteria;Chromatiales;Chromatiaceae;Thiophaeococcus sp. JA592,Bacteria;Firmicutes;Bacilli;Bacillales;anaerobic thermophile IC-BH;?,Bacteria;Actinobacteria;Actinobacteria;Micrococcales;Cellulomonadaceae;actinobacterium PY,Bacteria;Nitrospirae;Nitrospira;Nitrospirales;BMS9AB35;uncultured delta proteobacterium GR-296.I.86,Bacteria;Proteobacteria;Gammaproteobacteria;KI89A clade;gamma proteobacterium HdN1;?,Bacteria;Cyanobacteria;Cyanobacteria;SubsectionIV;FamilyI;Aphanizomenon flos-aquae 1477
0,0.0,0.0,0.000000,0.0,0.000000e+00,0.000013,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13478,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000001,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13479,0.0,0.0,0.000002,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000001,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13480,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13481,0.0,0.0,0.000003,0.0,6.862017e-07,0.000000,0.0,0.000000,0.000003,0.000042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9) # The seed was 'chosen' so test and training contain all labels: rn=3,4,8,9
print("train label deficit:",len(set(y)-set(y_train)),"test label deficit:",len(set(y)-set(y_test)))

train label deficit: 0 test label deficit: 0


In [10]:
class_counts = y_train.value_counts()
max_count = max(class_counts)
class_weights = {i:max_count/x for i,x in class_counts.iteritems()}
class_weights

{'Human_Gut': 1.0,
 'Human_Oral': 4.545454545454546,
 'Built': 6.033519553072626,
 'Human_Skin': 6.367924528301887,
 'Soil': 14.713896457765667,
 'Wastewater': 24.65753424657534,
 'Pig_Gut': 25.35211267605634,
 'Mouse_Gut': 33.333333333333336,
 'Human_Nose': 36.734693877551024,
 'Marine': 50.0,
 'Cat_Gut': 53.46534653465346,
 'Human_Vagina': 56.25,
 'Dog_Gut': 58.064516129032256,
 'Fresh_Water': 66.66666666666667,
 'Birds': 90.0,
 'Dairy': 245.45454545454547,
 'Bovine_Rumen': 257.14285714285717,
 'Food': 317.6470588235294,
 'Fermented': 337.5,
 'Composting': 385.7142857142857,
 'Root': 450.0,
 'Porifera': 600.0,
 'Phylloplane': 1350.0,
 'Hot_spring': 5400.0,
 'Fish_Skin': 5400.0}

In [17]:
# Optimize hyperparameters with grid_search
iseed = 67

cb_model = CatBoostClassifier(
    iterations=50,
    verbose=5,
    learning_rate=0.02,
    depth=5,
    eval_metric='MCC',
    random_seed=iseed,
    bagging_temperature=0.2,
    od_type='Iter',
    od_wait=100,
    class_weights=class_weights)

grid = {'learning_rate': [0.06, 0.1,0.14],
        'depth': [5, 6, 7],
        'l2_leaf_reg': [0.5, 1, 5]}

In [18]:
modelstart = time.time()
print(f"Starting model parameter optimization at {modelstart}")

grid_search_result = cb_model.grid_search(grid, X=X_train, y=y_train,verbose=5)
lr = grid_search_result['params']['learning_rate']
de = grid_search_result['params']['depth']
l2 = grid_search_result['params']['l2_leaf_reg']

print(f"Finished in {time.time() - modelstart}s")

Starting model parameter optimization at 1614658883.2858696

bestTest = 0.5391037775
bestIteration = 42

0:	loss: 0.5391038	best: 0.5391038 (0)	total: 2m 41s	remaining: 1h 9m 47s

bestTest = 0.6019455654
bestIteration = 49


bestTest = 0.6078289866
bestIteration = 49


bestTest = 0.535065066
bestIteration = 45


bestTest = 0.6080746047
bestIteration = 49


bestTest = 0.610279726
bestIteration = 49

5:	loss: 0.6102797	best: 0.6102797 (5)	total: 14m 55s	remaining: 52m 14s

bestTest = 0.6044197956
bestIteration = 49


bestTest = 0.6104627795
bestIteration = 47


bestTest = 0.5919108678
bestIteration = 46


bestTest = 0.6198825981
bestIteration = 49


bestTest = 0.645240218
bestIteration = 49

10:	loss: 0.6452402	best: 0.6452402 (10)	total: 32m 57s	remaining: 47m 56s

bestTest = 0.659178178
bestIteration = 48


bestTest = 0.638923497
bestIteration = 48


bestTest = 0.6517916713
bestIteration = 49


bestTest = 0.6602491771
bestIteration = 49


bestTest = 0.6351963205
bestIteration = 48

15:

Finished in 11520.567887067795s


In [19]:
iseed = 52

cb_model = CatBoostClassifier(
    iterations=200,
    verbose=5,
    learning_rate=lr,
    depth=de,
    l2_leaf_reg=l2,
    eval_metric='MCC',
    random_seed = iseed,
    bagging_temperature = 0.2,
    od_type='Iter',
    od_wait=100,
    class_weights=class_weights,
)

In [21]:
modelstart = time.time()
print(f"Starting training at {modelstart}")
cbmpf = cb_model.fit(X_train, y_train);
print(f"Training finished in {time.time() - modelstart}s")

Starting training at 1614712781.4332511
0:	learn: 0.1694427	total: 7.5s	remaining: 24m 52s
5:	learn: 0.3971195	total: 44.3s	remaining: 23m 52s
10:	learn: 0.5834309	total: 1m 21s	remaining: 23m 27s
15:	learn: 0.6813338	total: 1m 57s	remaining: 22m 30s
20:	learn: 0.7457870	total: 2m 34s	remaining: 21m 52s
25:	learn: 0.7893780	total: 3m 11s	remaining: 21m 19s
30:	learn: 0.8268042	total: 3m 44s	remaining: 20m 25s
35:	learn: 0.8515885	total: 4m 20s	remaining: 19m 46s
40:	learn: 0.8707013	total: 4m 57s	remaining: 19m 13s
45:	learn: 0.8811124	total: 5m 32s	remaining: 18m 34s
50:	learn: 0.8892213	total: 6m 7s	remaining: 17m 54s
55:	learn: 0.9017350	total: 6m 41s	remaining: 17m 12s
60:	learn: 0.9118171	total: 7m 18s	remaining: 16m 38s
65:	learn: 0.9196248	total: 7m 49s	remaining: 15m 53s
70:	learn: 0.9297423	total: 8m 25s	remaining: 15m 17s
75:	learn: 0.9363932	total: 8m 56s	remaining: 14m 36s
80:	learn: 0.9507879	total: 9m 34s	remaining: 14m 4s
85:	learn: 0.9747566	total: 10m 10s	remaining: 13

In [22]:
cbmpf

In [29]:
predictions = cbmpf.predict(X_test)
predictions_probs = cbmpf.predict_proba(X_test)

In [33]:
print(predictions[:20])
print(y_test[:20])
print(predictions_probs[:10])
# list(zip(predictions[:10], predictions_probs[:10]))

[['Human_Gut']
 ['Human_Skin']
 ['Soil']
 ['Human_Oral']
 ['Human_Nose']
 ['Soil']
 ['Human_Gut']
 ['Human_Gut']
 ['Marine']
 ['Human_Gut']
 ['Human_Gut']
 ['Human_Gut']
 ['Human_Skin']
 ['Human_Oral']
 ['Dog_Gut']
 ['Human_Gut']
 ['Human_Gut']
 ['Human_Gut']
 ['Human_Gut']
 ['Human_Skin']]
1768      Human_Gut
5374     Human_Skin
36             Soil
2329     Human_Oral
2070     Human_Nose
1021           Soil
12304     Human_Gut
11147     Human_Gut
10389        Marine
9154      Human_Gut
10315     Human_Gut
252       Human_Gut
1452     Human_Skin
11031    Human_Oral
1093        Dog_Gut
5761      Human_Gut
12216     Human_Gut
3322      Human_Gut
13406     Human_Gut
7902     Human_Skin
Name: Habitat_type, dtype: object
[[8.25211173e-01 1.43257107e-02 1.63321556e-02 1.24455798e-02
  1.00783908e-02 1.44927632e-02 1.54765867e-03 7.34471560e-03
  1.64544047e-02 2.72343385e-03 2.14879211e-02 1.49132924e-02
  1.08888688e-02 5.72431507e-04 1.04682616e-02 1.14721629e-03
  3.03453399e-03 2.2158458